# Necessary Imports 

In [2]:
!pip install contractions
!pip install pyvi
import torch
from torch import nn, optim
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.metrics import bleu_score
from torchtext.data.utils import get_tokenizer
from pyvi import ViTokenizer
import re
import html
import contractions
import requests
from typing import Iterator,List, Optional

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 40.1 MB/s 
     |████████████████████████████████| 106 kB 73.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.5 MB 25.1 MB/s 
     |████████████████████████████████| 965 kB 59.6 MB/s 


# 2. Data Preparation & Pre-processing

In [3]:
!python -m spacy download en_core_web_sm --quiet

     |████████████████████████████████| 12.8 MB 27.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [62]:
class Data:
  def __init__(self, url_en:str, url_vi:str):
    Data.check_dict = { # bổ xung
      ' \'s': '\'s',
      '& lt ;': '<',
      '& gt ;': '>',
      "<[^<]+>":'',
      ' +': ' ',
    }
    Data.tokenizer = {
      'vi': lambda text: list(map(lambda word: re.sub('_', ' ', word), ViTokenizer.tokenize(text).split())),
      'en': get_tokenizer('spacy', language='en_core_web_sm')
    }
    data_en = requests.get(url_en).text.strip().splitlines()
    data_vi = requests.get(url_vi).text.strip().splitlines()
    self.__data_en = [self.__text_preprocessing(en, 'en') for en in data_en]
    self.__data_vi = [self.__text_preprocessing(vi, 'vi') for vi in data_vi]

  def __text_preprocessing(self, text: str, language: str = 'en'):
    text = html.unescape(text)
    for pattern, repl in Data.check_dict.items():
      text = text.lower()
      text = re.sub(pattern, repl, text)

    if language == 'en':
      text = re.sub(' +', ' ', contractions.fix(text))
      return self.tokenizer['en'](text)
    return self.tokenizer['vi'](text)

  @property
  def en(self):
    return self.__data_en

  @property
  def vi(self):
    return self.__data_vi
  


In [63]:
url = "https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/"

train_data = Data(url +'train.en',url +'train.vi')
# val_data = Data(url + 'tst2012.en',url + 'tst2012.vi')
test_data = Data(url + 'tst2013.en',url + 'tst2013.vi')

In [68]:
for text in train_data.vi[:5]:
  print(text)

['khoa học', 'đằng', 'sau', 'một', 'tiêu đề', 'về', 'khí hậu']
['trong', '4', 'phút', ',', 'chuyên gia', 'hoá học', 'khí quyển', 'rachel', 'pike', 'giới thiệu', 'sơ lược', 'về', 'những', 'nỗ lực', 'khoa học', 'miệt mài', 'đằng', 'sau', 'những', 'tiêu đề', 'táo bạo', 'về', 'biến đổi', 'khí hậu', ',', 'cùng', 'với', 'đoàn', 'nghiên cứu', 'của', 'mình', '-', '-', 'hàng', 'ngàn', 'người', 'đã', 'cống hiến', 'cho', 'dự án', 'này', '-', '-', 'một', 'chuyến', 'bay', 'mạo hiểm', 'qua', 'rừng già', 'để', 'tìm kiếm', 'thông tin', 'về', 'một', 'phân tử', 'then chốt', '.']
['tôi', 'muốn', 'cho', 'các', 'bạn', 'biết', 'về', 'sự', 'to lớn', 'của', 'những', 'nỗ lực', 'khoa học', 'đã', 'góp phần', 'làm nên', 'các', 'dòng', 'tít', 'bạn', 'thường', 'thấy', 'trên', 'báo', '.']
['có', 'những', 'dòng', 'trông', 'như', 'thế', 'này', 'khi', 'bàn', 'về', 'biến đổi', 'khí hậu', ',', 'và', 'như', 'thế', 'này', 'khi', 'nói', 'về', 'chất lượng', 'không khí', 'hay', 'khói', 'bụi', '.']
['cả', 'hai', 'đều', 'là', '

In [70]:
class Language:
  def __init__(self, data: Data, min_freq:int = 1):
    specials = ["<unk>", "<pad>", "<sos>", "<eos>"]

    self.__vocab_en = build_vocab_from_iterator(self.__yield_tokens(data.en,'en'), min_freq, specials)
    self.__vocab_en.set_default_index(0)

    self.__vocab_vi = build_vocab_from_iterator(self.__yield_tokens(data.vi,'vi'), min_freq, specials)
    self.__vocab_vi.set_default_index(0)
  
  def __yield_tokens(self, data:List[str] , language:str = 'en'):
    for line in data:
      yield line

  @property
  def en(self):
    return self.__vocab_en

  @property
  def vi(self):
    return self.__vocab_vi

  def text_pipeline(self, data:List[str], language:str = 'en') -> List[str]:
    if language == 'en':
      return [vocab.en.lookup_tokens([2,*vocab.en.lookup_indices(line),3]) for line in data]
    if language == 'vi':
      return [vocab.vi.lookup_tokens([2,*vocab.vi.lookup_indices(line),3]) for line in data]

In [71]:
vocab = Language(train_data,3)

In [72]:
print(f"Unique tokens in source (en) vocabulary: {len(vocab.en)}")
print(f"Unique tokens in target (vi) vocabulary: {len(vocab.vi)}")

Unique tokens in source (en) vocabulary: 21928
Unique tokens in target (vi) vocabulary: 16085


In [73]:
#vector
train_en_prep = vocab.text_pipeline(train_data.en,'en')
train_vi_prep = vocab.text_pipeline(train_data.vi,'vi')
test_en_prep = vocab.text_pipeline(test_data.en,'en')
test_vi_prep = vocab.text_pipeline(test_data.vi,'vi')

In [74]:
for i in range(5):
  print(f'train en: {train_en_prep[i]}')
  print(f'train vi: {train_vi_prep[i]}')
  print(f'test en: {test_en_prep[i]}')

train en: ['<sos>', 'rachel', 'pike', ':', 'the', 'science', 'behind', 'a', 'climate', 'headline', '<eos>']
train vi: ['<sos>', 'khoa học', 'đằng', 'sau', 'một', 'tiêu đề', 'về', 'khí hậu', '<eos>']
test en: ['<sos>', 'when', 'i', 'was', 'little', ',', 'i', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'i', 'grew', 'up', 'singing', 'a', 'song', 'called', '"', 'nothing', 'to', 'envy', '.', '"', '<eos>']
train en: ['<sos>', 'in', '4', 'minutes', ',', 'atmospheric', 'chemist', 'rachel', 'pike', 'provides', 'a', 'glimpse', 'of', 'the', 'massive', 'scientific', 'effort', 'behind', 'the', 'bold', 'headlines', 'on', 'climate', 'change', ',', 'with', 'her', 'team', '--', 'one', 'of', 'thousands', 'who', 'contributed', '--', 'taking', 'a', 'risky', 'flight', 'over', 'the', 'rainforest', 'in', 'pursuit', 'of', 'data', 'on', 'a', 'key', 'molecule', '.', '<eos>']
train vi: ['<sos>', 'trong', '4', 'phút', ',', 'chuyên gia', 'hoá học', 'khí quyển', 'rachel', 'p

In [75]:
train_data = list(zip(train_en_prep,train_vi_prep))
train_data.sort(key = lambda x: (len(x[0]), len(x[1])))
test_data = list(zip(test_en_prep, test_data.en, test_data.vi))

In [76]:
for i in range(5):
  print(train_data[i])

for i in range(5):   
  print(test_data[i])

(['<sos>', '<eos>'], ['<sos>', '<eos>'])
(['<sos>', '<eos>'], ['<sos>', '<eos>'])
(['<sos>', '<eos>'], ['<sos>', '<eos>'])
(['<sos>', '<eos>'], ['<sos>', '<eos>'])
(['<sos>', '<eos>'], ['<sos>', '<eos>'])
(['<sos>', 'when', 'i', 'was', 'little', ',', 'i', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'i', 'grew', 'up', 'singing', 'a', 'song', 'called', '"', 'nothing', 'to', 'envy', '.', '"', '<eos>'], ['when', 'i', 'was', 'little', ',', 'i', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'i', 'grew', 'up', 'singing', 'a', 'song', 'called', '"', 'nothing', 'to', 'envy', '.', '"'], ['khi', 'tôi', 'còn', 'nhỏ', ',', 'tôi', 'nghĩ', 'rằng', 'bắctriều', 'tiên', 'là', 'đất nước', 'tốt', 'nhất', 'trên', 'thế giới', 'và', 'tôi', 'thường', 'hát', 'bài', '"', 'chúng ta', 'chẳng', 'có', 'gì', 'phải', 'ghen tị', '.', '"'])
(['<sos>', 'and', 'i', 'was', 'very', 'proud', '.', '<eos>'], ['and', 'i', 'was', 'very', 'proud', '.

In [77]:
def make_batch(data, batchsize = 32):
  bb = []
  ben = []
  bvi = []
  for en, vi in data: 
    ben.append(en)
    bvi.append(vi)
    if len(ben) >= batchsize:
      bb.append((ben, bvi))
      ben = []
      bvi = []
  if len(ben) > 0:
    bb.append((ben, bvi))
  return bb


In [78]:
BATCH_SIZE = 32
train_data = make_batch(train_data, BATCH_SIZE)


In [79]:
for i in range(5):
  print(train_data[i])


([['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>']], [['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>'], ['<sos>', '<eos>

In [80]:
def padding_batch(b):
  maxlen = max([len(x) for x in b])
  for tkl in b:
    for i in range(maxlen - len(tkl)):
      tkl.append('<pad>')

def padding(bb):
  for ben, bvi in bb:
    padding_batch(ben)
    padding_batch(bvi)

padding(train_data)

In [81]:
train_data = [([vocab.en.lookup_indices(en) for en in ben],[vocab.vi.lookup_indices(vi)for vi in bvi]) for ben, bvi in train_data]
test_data = [(vocab.en.lookup_indices(enprep), en, vi) for enprep, en, vi in test_data]

In [82]:
for i in range (3): 
  print(train_data[i]) 
for i in range (3): 
  print(test_data[i]) 

([[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3]], [[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3]])
([[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3]], [[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3],

In [83]:
MODEL_NAME = "LSTM.model"
EPOCH = 10
LR = 0.0001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [84]:
from torch.nn.functional import cross_entropy
class LSTM(torch.nn.Module):
  def __init__(self, vocablist_x, vocabidx_x, vocablist_y, vocabidx_y):
    super(LSTM, self).__init__()

    self.encemb = torch.nn.Embedding(len(vocablist_x), 256, padding_idx = vocabidx_x['<pad>'])
    self.dropout = torch.nn.Dropout(0.5)
    self.enclstm = torch.nn.LSTM(256,516,2,dropout=0.5)
    
    self.decemb = torch.nn.Embedding(len(vocablist_x), 256, padding_idx = vocabidx_y['<pad>'])
    self.declstm = torch.nn.LSTM(256,516,2,dropout=0.5)
    self.decout = torch.nn.Linear(516, len(vocabidx_y))
  
  def forward(self,x):
    x, y = x[0], x[1]
    # print(x.size())
    # print(y.size())

    e_x = self.dropout(self.encemb(x))
    
    outenc,(hidden,cell) = self.enclstm(e_x)

    n_y=y.shape[0]
    loss = torch.tensor(0.,dtype=torch.float32).to(DEVICE)
    for i in range(n_y-1):
      input = y[i]
      input = input.unsqueeze(0)
      input = self.dropout(self.decemb(input))
      outdec, (hidden,cell) = self.declstm(input,(hidden,cell))
      output = self.decout(outdec.squeeze(0))
      input = y[i+1]
      loss += cross_entropy(output, y[i+1])
    return loss

  def evaluate(self,x,vocablist_y,vocabidx_y):
    e_x = self.dropout(self.encemb(x))
    outenc,(hidden,cell)=self.enclstm(e_x)
    
    y = torch.tensor([vocabidx_y['<cls>']]).to(DEVICE)
    pred=[]
    for i in range(30):
      input = y
      input = input.unsqueeze(0)
      input = self.dropout(self.decemb(input))
      outdec,(hidden,cell)= self.declstm(input,(hidden,cell))
      output = self.decout(outdec.squeeze(0))  
      pred_id = output.squeeze().argmax().item()
      if pred_id == vocabidx_y['<eos>']:
        break
      pred_y = vocablist_y[pred_id]
      pred.append(pred_y)
      y[0]=pred_id
      input=y
    return pred  

In [85]:
def train_LMST():
  model = LSTM(vocab.en.get_itos(), vocab.en, vocab.vi.get_itos(), vocab.vi).to(DEVICE)
  optimizer = torch.optim.Adam(model.parameters(), lr=LR) 
  for epoch in range(EPOCH):
    loss = 0
    step = 0
    for ben, bvi in train_data:
      ben = torch.tensor(ben, dtype=torch.int64).transpose(0,1).to(DEVICE) 
      bvi = torch.tensor(bvi, dtype=torch.int64).transpose(0,1).to(DEVICE)
      optimizer.zero_grad()
      batchloss = model((ben, bvi))
      batchloss.backward()
      optimizer.step() 
      loss = loss + batchloss.item()
      # if step % 100 == 0:
      #   print("step:", step, "batch loss:", batchloss.item())
      # step += 1
    print("epoch", epoch, ": loss", loss)
  torch.save(model.state_dict(), MODEL_NAME)

In [ ]:
train_LMST()

epoch 0 : loss 525552.2081832886
epoch 1 : loss 487123.3473596573
epoch 2 : loss 462455.4591603279
epoch 3 : loss 445120.3843970299
epoch 4 : loss 431497.23986268044
epoch 5 : loss 420206.5520154834


In [ ]:
from torchtext.data.metrics import bleu_score
def test_LMST():
  model = LSTM(vocab.en.get_itos(), vocab.en, vocab.vi.get_itos(), vocab.vi).to(DEVICE)
  model.load_state_dict(torch.load(MODEL_NAME))
  model.eval()
  ref = []
  pred = []
  for enprep, en, vi in test_data:
    input = torch.tensor([enprep], dtype=torch.int64).transpose(0, 1).to(DEVICE)
    p=model.evaluate(input, vocab.vi.get_itos(), vocab.vi)
    print("INPUT", en)
    print("REF", vi)
    print("MT", p)
    ref.append([vi])
    pred.append(p)
  print("total:", len(test_data)) 
  bleu = bleu_score(pred, ref,0,[1])
  print("bleu:", bleu)

In [ ]:
test_LMST()
